## My Classifier test score
- Binary Classifier for MNIST: 92.89%
- Multi Class Classifier for MNIST: 93.56%
- Multi Class Classifier for CIFAR10: 51.47%

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

c:\Users\jwhau\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Normalize data with mean=0.5, std=1.0
mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,))
])

# download path 정의
download_root = './MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [3]:
# Exercise 1-2
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html
class MultiClassMLP(nn.Module) :
    def __init__(self, in_dim, out_dim, inter_dim=24*24) :
        super().__init__()
        self.in_dim = in_dim
        self.linear1 = nn.Linear(in_dim, inter_dim, bias=True)
        self.act1 = nn.Sigmoid()
        self.linear2 = nn.Linear(inter_dim, inter_dim, bias=True)
        self.act2 = nn.Sigmoid()
        self.linear3 = nn.Linear(inter_dim, out_dim, bias=True)
        
    def forward(self, x) :
        # assert x.size()[2]*x.size()[3] == self.in_dim, "x.size()[2]*x.size()[3] should be {}".format(self.in_dim)
        x = x.reshape(-1, self.in_dim)
        z = self.linear1(x)
        z = self.act1(z)
        z = self.linear2(z)
        z = self.act2(z)
        z = self.linear3(z)
        return z

In [4]:
# MultiClass Classfication
STUDENT_model = MultiClassMLP(28*28, 10)

In [5]:
# Please change "STUDENT_model" with your NN model class's instance name
total_params = sum(p.numel() for p in STUDENT_model.parameters())

print("Number of Parameters: ", total_params)

Number of Parameters:  790282


In [6]:
train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [7]:
# option 값 정의
batch_size = 32
dim = 28*28
learning_rate = 1e-1

train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [8]:
my_loss = nn.CrossEntropyLoss()

print(my_loss(torch.Tensor([1, 2, 3]), torch.Tensor([4, 5, 6])))
print(my_loss(torch.Tensor([1, 2, 3]), torch.Tensor([4, 2, 3])))

tensor(19.1141)
tensor(13.6685)


In [9]:
optimizer = optim.SGD(STUDENT_model.parameters(), learning_rate)

for epoch in range(7):
    STUDENT_model.train()
    epoch_loss = 0
    
    for i, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        output = STUDENT_model(data)
        
        loss = my_loss(output, target)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f'Epoch: {epoch} | Train Loss: {epoch_loss / len(train_loader)}')

Epoch: 0 | Train Loss: 1.39730984582901
Epoch: 1 | Train Loss: 0.44338167666594186
Epoch: 2 | Train Loss: 0.363048623863856
Epoch: 3 | Train Loss: 0.3238696164906025
Epoch: 4 | Train Loss: 0.2914444414893786
Epoch: 5 | Train Loss: 0.26659294713238874
Epoch: 6 | Train Loss: 0.2425361001521349


In [10]:
''' Test for train_dataset '''

image_size = 28*28
#image_size = 3*32*32 # for CIFAR dataset
STUDENT_model.eval()
batch_size = 32

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
count = 0
for image,label in train_loader :
    label = label
    prediction = torch.argmax(STUDENT_model(image.view(len(image),image_size)),dim=1)
    count += (prediction == label).sum()
        
print('Test for train_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(train_dataset.targets), count/len(train_dataset.targets) * 100. ) )    

''' Test for test_dataset '''
count = 0
wrong_answer = []
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
for image,label in test_loader :
    label = label
    prediction = torch.argmax(STUDENT_model(image.view(len(image),image_size)),dim=1)
    count += (prediction == label).sum()
    
#     else:
#         wrong_answer += [i]   
        
print('Test for test_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(test_dataset.targets), count/len(test_dataset.targets) * 100. ) )    

Test for train_dataset: correctly matched/total: 55984/60000 accuracy: 93.31%

Test for test_dataset: correctly matched/total: 9321/10000 accuracy: 93.21%

